# How to execute and import a jupyter notebook?

Include this header script in your notebooks to find more answers.

How to use inheritence in jupyter notebooks?

How to make a class out of jupter notebooks?

How to use jupyter notebooks as services?



How to stream json using a {match} function?

In [1]:
// initialize
var path = require('path');
var fs = require('fs');
var Fiber = require('fibers');
var Future = require('fibers/future');

// provide a function for importing any notebook as a module and executing it
var getCellsOrDirectory = (filename) => {
    const fname = filename.replace('.ipynb', '');
    if (fs.existsSync(fname)
        && fs.statSync(fname).isDirectory()) {
        const files = fs.readdirSync(fname)
            .sort((a, b) => parseInt(a.replace(/[^0-9]/ig, '')) - parseInt(b.replace(/[^0-9]/ig, ''))) //sort numerically
            .filter(f => f.indexOf('.js') > -1);
        return files.map(f => fs.readFileSync(path.join(fname, f)).toString());
    }
    else {
        return getCells(filename, ['javascript', 'code'])
            .map((c, i) => ({
                fresher: c.source.join(''),
                id: path.basename(filename) + '[' + i + ']',
                filename: filename,
                runInNewContext: (ctx) => runInNewContext(c.source.join(''), Object.assign(ctx, {
                    __filename: filename,
                }), {}, false)
            }));
    }
};

// output nothing here, cached version of this function is assigned below.
if (typeof imported !== 'object' || imported === null) {
    var imported = {};
}

// How to test if a notebook has already been imported?
var importNotebook = (notebook, ctx = {}) => {
    if (typeof notebook === 'undefined') {
        return Promise.resolve({});
    }
    var cells;
    // TODO: do Core notebooks synchronously?
    if(typeof notebook === 'string' && fs.existsSync(path.resolve(notebook))) {
        // ONLY RETURN CACHE?
        if (typeof imported[notebook] !== 'undefined' && path.resolve(notebook) === path.resolve('interpret questions.ipynb')) {
            return imported[notebook];
        }
        cells = getCellsOrDirectory(path.resolve(notebook));
    } else {
        cells = interpret(notebook);
        if( typeof cells.fresher !== 'undefined') {
            return (imported[notebook] = cells.runInNewContext(ctx));
        }
    }
    
    return (imported[notebook] = cells.reduce((obj, c, i) => {
        var result = c.runInNewContext(ctx);
        if (typeof result === 'function') {
            obj[cells[i].id] = result;
            obj[result.name] = obj[cells[i].id];
        }
        obj[i] = result;
        Object.assign(ctx, obj);
        return obj;
    }, {}));
};

// $$.done(); for our dumb parser



In [ ]:
// initialize
var fs = require('fs');var JSONStream = require('JSONStream');var jp = require('jsonpath');

/*var streamJson = (file, parse, match = ((c) => true)) => {
    var stream = fs.createReadStream(file)        .pipe(JSONStream.parse(parse));
    stream.on('data', (m) => match(m));
    console.log(file);
    // create a promise out of this stream    return new Promise((resolve, reject) => {        stream.on('error', e => reject(e));        stream.on('close', () => resolve());    });};
*/

var streamJson = (file, path, match = ((c) => true)) => {
    
    const json = JSON.parse(fs.readFileSync(file).toString());
    const nodes = jp.nodes(json, '$..*');
    return nodes.filter(n => typeof n.value === 'object' && n.value !== null).forEach(n => match({
        path: n.path.slice(1),
        value: n.value}));
};
// $$.done() for our dumb parser


How to parse cells of {types} from a notebook?



In [ ]:
// initialize// How to use getCells?var getCells = (notebook, types = ['*', 'code']) => {
    var kernel;
    var cells = [];
    streamJson(notebook, [true, {emitPath: true}], (match) => {
        if (match.path[0] === 'metadata'
            && match.path[1] === 'kernelspec') {
            kernel = match.value;
        } else if (match.path[0] === 'cells'
            && types.indexOf(match.value.cell_type) > -1) {
            cells[cells.length] = match.value;
        }
    });
    if (types.indexOf(kernel.language) === -1
        && types.indexOf('*') === -1) {
        console.log('tried to import ' + types + JSON.stringify(kernel));
        return [];
    }
    return cells.map(c => Object.assign(c, {language: kernel.language}));
};// $$.done() for our dumb parser

How to run all promises sequentially?



In [ ]:
// initializevar runAllPromises = (promises) => {    return promises.reduce((promise, func) => {        return promise.then(result => {            return (typeof func == 'function'                ? (new Promise(func)) : Promise.resolve(func))                .then(Array.prototype.concat.bind(result));        });    }, Promise.resolve([]));};// $$.done() for our dumb parser

How to load interpreter?

How to search notebooks?


In [ ]:

// initialize
var fs = require('fs');
var path = require('path');

var fileMode = (s, compare) => compare.substr(0, path.basename(s).split('[')[0].length) === s;

var fuseSearch, interpretObject;
var interpret = (s) => {
    if(typeof fuseSearch === 'undefined') {
        var interpreter = importNotebook('interpret all notebooks.ipynb')[1];
        fuseSearch = interpreter.fuseSearch;
        interpretObject = interpreter.interpretObject;
    }
    var searches = (typeof s === 'string' ? [s] : s)
        .map(q => fuseSearch(q));
    if (searches.length === 0
        || (typeof s === 'string'
            && typeof searches[0] === 'undefined')
        || (searches.length === 1
            && searches[0].length === 0)) {
        throw 'Nothing found for ' + JSON.stringify(s).substr(0, 200);
    }
    searches = typeof s === 'string' ? searches[0] : searches;
    var interpreted;
    // TODO: whole file mode or [filename.ipynb]?
    interpreted = typeof s === 'string'
        ? interpretObject(fileMode(s, searches[0]) ? searches : [searches[0]])
        : interpretObject(searches.map((r, i) => fileMode(s[i], r[0]) ? r : r[0]));
    
    const results = interpreted.map(r => {
        if (typeof r === 'undefined') {
            return r;
        }
        // must have a unique id for each unique cell so that
        // individual cells can serve as modules as well as notebooks
        // adding the cell id as a part of the filename
        r.runInNewContext = (ctx) => runInNewContext(
            r.fresher,
            Object.assign(ctx || {}, {
                useCache: false,
                __filename: r.filename
            }),
            {filename: path.join(path.dirname(r.filename), r.id)}, false);
        return r;
    });
    return typeof s === 'string' && !fileMode(s, searches[0]) ? results[0] : results;
};

// $$.done() for our dumb parser



Import arbitrary code in to the current context?

In [ ]:
// initialize
var Module = require('module').Module;
var path = require('path');
var requireCode = (code, pathToCode, tmpGlobal) => {

    // must have a new name for every generation otherwise cache will be returned
    var filepath = path.resolve(process.cwd(), pathToCode);
    var dirname = path.dirname(filepath);

    var cachedModule = Module._cache[filepath];
    if (cachedModule && tmpGlobal.useCache !== false) {
        tmpGlobal.module = cachedModule;
        return cachedModule.exports;
    }

    var mod = new Module(filepath, global.module);
    tmpGlobal.module = mod;
    Module._cache[filepath] = mod;

    mod.filename = filepath;
    mod.paths = Module._nodeModulePaths(dirname);

    mod._compile('module.exports=(function(ctx) { Object.assign(global, ctx);\n' + code + '\n})', filepath);
    mod.loaded = true;
    mod.exports(tmpGlobal);
    tmpGlobal.exports = mod.exports;
    return mod.exports;
};

// $$.done() for our dumb parser



Import arbitrary code in to a new context?

In [ ]:
// initialize

var Module = require('module').Module;
var path = require('path');
var vm = require('vm');

var runInNewContext = (code, ctx, options, isModule = true) => {
    var tmpGlobal = ctx || {};
    tmpGlobal.global = tmpGlobal;
    Object.assign(tmpGlobal, ctx);
    Object.assign(tmpGlobal, {
        'console': console,
        'process': process,
        'Promise': Promise,
        'Buffer': Buffer,
        'clearImmediate': clearImmediate,
        'clearInterval': clearInterval,
        'clearTimeout': clearTimeout,
        'setImmediate': setImmediate,
        'setInterval': setInterval,
        'setTimeout': setTimeout,
        '__filename': path.basename(ctx.__filename),
        '__dirname': path.dirname(ctx.__filename),
        'require': require
    });
    Object.assign(options || {}, {lineOffset: -1});
    if (!isModule) {
        tmpGlobal.module = {};
    }
    if(isModule) {
        var result = requireCode(code, path.join(ctx.__dirname, ctx.__filename), tmpGlobal);
        return tmpGlobal.global.module.exports || result;
    } else {
        var result = vm.runInNewContext('"use strict";\n' + code, tmpGlobal, options);
        module.paths = Module._nodeModulePaths(path.join(ctx.__dirname, ctx.__filename));
        return tmpGlobal.global.module.exports || result;
    }
};

// $$.done() for our dumb parser


In [ ]:
// initialize

if (typeof imported !== 'object' || imported === null) {
    var imported = {};
}

imported['import notebook.ipynb'] = {
    'import notebook.ipynb[2]': streamJson,
    'streamJson': streamJson,
    'import notebook.ipynb[3]': getCells,
    'getCells': getCells,
    'import notebook.ipynb[4]': runAllPromises,
    'runAllPromises': runAllPromises,
    'import notebook.ipynb[7]': importNotebook,
    'import': importNotebook,
    'import notebook.ipynb[8]': imported,
    'imported': imported,
    'import notebook.ipynb[9]': interpret,
    'interpret': interpret,
    'import notebook.ipynb[11]': runInNewContext,
    'runInNewContext': runInNewContext
};

module.exports = imported['import notebook.ipynb'];

// $$.done() for our dumb parser



These are some other useful notebooks you may want to import.

How to train a chatbot using jupyter services?

What is code completion for jupter notebooks?

